In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2%}'.format
import numpy as np
df = pd.read_csv('summary.csv', names=['ga', 'problem', 'ell', 'nfe_mean', 'nfe_std'], skiprows=1)
df = df[df['ga'] != 'dsmga2-tos-ftf_mfN']
df = df[df['ga'] != 'dsmga2-tos-ftf_cntN']

In [2]:
def examine(problem, ell, ga=None, neat=False):
    global df
    cond = np.logical_and(df['problem'] == problem, df['ell'] == ell)
    if ga:
        cond = np.logical_and(cond, df['ga'] == ga)
    
    res = df[cond].sort_values('nfe_mean', ignore_index=True)
    if neat:
        res['nfe_mean'] = (res['nfe_mean'] // 1000).astype(int)
        res['nfe_std'] = (res['nfe_std'] // 1000).astype(int)
    return res

In [3]:
examine(problem='mktrap', ell=400, ga=None, neat=True)

,ga,problem,ell,nfe_mean,nfe_std
0,TFF_MEDIAN,mktrap,400,45,3
1,TFF_MAX,mktrap,400,46,1
2,TFF_MEAN,mktrap,400,46,1
3,TTF_MEAN_MOE,mktrap,400,48,3
4,TTT_MEAN_MOG_S,mktrap,400,48,2
5,dsmga2-tos-ftf_cntG,mktrap,400,49,4
6,TTT_MEAN_MOE_S,mktrap,400,49,5
7,dsmga2-tos-ftf_moE,mktrap,400,49,5
8,dsmga2-tos-ftt_moG,mktrap,400,49,2
9,dsmga2-tos-fft_moG,mktrap,400,49,4


In [6]:
examine(problem='nk', ell=400, ga=None, neat=True)

,ga,problem,ell,nfe_mean,nfe_std
0,TTT_MEAN_CE_S,nk,400,596,290
1,TTF_MEAN_CE,nk,400,606,310
2,TFF_MEDIAN,nk,400,630,313
3,TTF_MEAN_MOE,nk,400,636,325
4,TFF_MEAN,nk,400,636,338
5,TTT_MEAN_MOE_S,nk,400,638,329
6,TTT_MEAN_MOG_S,nk,400,656,313
7,TTF_MEAN_MOG,nk,400,658,325
8,TTT_MEAN_CG_S,nk,400,660,346
9,TTF_MEAN_CG,nk,400,670,369


In [5]:
examine(problem='sat', ell=100, ga=None, neat=True)

,ga,problem,ell,nfe_mean,nfe_std
0,TTT_MEAN_MOE_S,sat,100,159,217
1,TTT_MEAN_MOG_S,sat,100,162,237
2,TTF_MEAN_CG,sat,100,169,241
3,TTT_MEAN_CG_S,sat,100,170,229
4,TTT_MEAN_CE_S,sat,100,171,237
5,TTF_MEAN_MOG,sat,100,171,246
6,TTF_MEAN_MOE,sat,100,172,231
7,dsmga2-tos-fff,sat,100,177,252
8,TTF_MEAN_CE,sat,100,178,250


In [4]:
examine(problem='cyctrap', ell=400, ga=None, neat=True)

,ga,problem,ell,nfe_mean,nfe_std
0,TTT_MEAN_CE_S,cyctrap,400,97,13
1,TTF_MEAN_CE,cyctrap,400,97,9
2,dsmga2-tos-ftf_cntE,cyctrap,400,98,14
3,TTT_MEAN_CG_S,cyctrap,400,108,13
4,dsmga2-tos-ftf_moE,cyctrap,400,108,14
5,dsmga2-tos-ftt_moE,cyctrap,400,110,16
6,TTT_MEAN_MOE_S,cyctrap,400,113,17
7,TTF_MEAN_CG,cyctrap,400,116,21
8,dsmga2-tos-ftf_cntG,cyctrap,400,117,20
9,dsmga2-tos-fft_moG,cyctrap,400,119,10


In [3]:
examine(problem='ftrap', ell=480, ga=None, neat=True)

,ga,problem,ell,nfe_mean,nfe_std
0,dsmga2-tos-ftf_cntG,ftrap,480,177,1
1,TTF_MEAN_CG,ftrap,480,178,5
2,dsmga2-tos-ftt_count,ftrap,480,180,6
3,dsmga2-tos-ftf_cntE,ftrap,480,200,2
4,TTF_MEAN_CE,ftrap,480,202,1
5,TTT_MEAN_CG_S,ftrap,480,203,5
6,TTF_MEAN_MOG,ftrap,480,215,3
7,dsmga2-tos-ftf_moN,ftrap,480,222,3
8,TTT_MEAN_CE_S,ftrap,480,223,3
9,TFF_MEDIAN,ftrap,480,226,3


In [22]:
examine(problem='spin', ell=400, ga=None, neat=True)

,ga,problem,ell,nfe_mean,nfe_std
0,TTT_MEAN_CG_S,spin,400,160,68
1,TTF_MEAN_CG,spin,400,164,80
2,TTF_MEAN_MOG,spin,400,167,62
3,TTF_MEAN_CE,spin,400,171,83
4,TTT_MEAN_CE_S,spin,400,172,60
5,TTF_MEAN_MOE,spin,400,185,74
6,TTT_MEAN_MOG_S,spin,400,192,75
7,TTT_MEAN_MOE_S,spin,400,194,67
8,dsmga2-tos-fff,spin,400,205,105


In [5]:
examine(problem='maxcut', ell=100, ga=None, neat=True)

,ga,problem,ell,nfe_mean,nfe_std
0,TTF_MEAN_CG,maxcut,100,273,218
1,TTF_MEAN_MOG,maxcut,100,275,289


In [4]:
d_list = []

problem_list = ['mktrap', 'ftrap', 'cyctrap', 'nk', 'spin', 'sat']
ell_list = [400, 480, 400, 400, 400, 100]
ga_list = ['TTT_MEAN_CG_S', 'TTT_MEAN_CE_S', 'TTT_MEAN_MOE_S', 'TTT_MEAN_MOG_S', 'dsmga2-tos-fff']

for problem, ell in zip(problem_list, ell_list):
    # base = df[df['ga'] == 'dsmga2-tos-fff']['nfe_mean'].values
    tmp_df = df[(df['problem']==problem) & (df['ell']==ell)].loc[df['ga'].isin(ga_list)]
    base = tmp_df[tmp_df['ga'] == 'dsmga2-tos-fff']['nfe_mean'].values
    k = tmp_df['ga'].values
    v = (tmp_df['nfe_mean'].values - base)/base
    d_list.append(dict(zip(k, v.T)))

df_list = []

for d, problem in zip(d_list, problem_list):
    df_list.append(pd.DataFrame(data=d.values(), index=d.keys(), columns=[problem]))

df_rank = pd.concat(df_list, axis=1)
df_rank['mean'] = df_rank.iloc[:,:4].mean(axis=1)
df_rank['mean'].sort_values
df_rank = df_rank.sort_values('mean')
df_rank

,mktrap,ftrap,cyctrap,nk,spin,sat,mean
TTT_MEAN_CG_S,0.61%,-14.79%,-10.44%,-2.78%,-21.69%,-4.26%,-6.85%
TTT_MEAN_CE_S,12.14%,-6.09%,-19.18%,-12.15%,-15.83%,-3.48%,-6.32%
TTT_MEAN_MOE_S,-6.29%,11.23%,-5.82%,-5.94%,-5.33%,-10.49%,-1.70%
dsmga2-tos-fff,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
TTT_MEAN_MOG_S,-8.83%,6.89%,11.31%,-3.30%,-6.16%,-8.87%,1.52%
